## Setup of the environnement

In [35]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, explode, json_tuple, regexp_replace, udf
from pyspark.sql.functions import sum as col_sum
from pyspark.sql.types import BooleanType, StructType, StructField, StringType, IntegerType, FloatType

from math import radians, cos, sin, asin, sqrt
from typing import List

import re
import os

## Find the center coordinates of the villages

In [52]:
data = sc.textFile("data/zipcodes.csv")
data = data.map(lambda x: re.sub(r' \(.*\)', '', x))
data = data.map(lambda x: re.sub(r"[^\S\n\t]+", ' ', x))
data = data.map(lambda x: x.split(';'))

schema = StructType([
    StructField('postal code', StringType(), False),
    StructField('Village', StringType(), False),
    StructField('lat_center', StringType(), False),
    StructField('lon_center', StringType(), False),
    StructField('url', StringType(), False)
])


data = data.toDF(schema=schema).drop("url")
data = data.withColumn("lat_center", data['lat_center'].cast(FloatType()))
data = data.withColumn("lon_center", data['lon_center'].cast(FloatType()))

centers = data.where(col("lat_center").isNotNull() & col("lon_center").isNotNull())

centers.show()

+-----------+--------------------+----------+----------+
|postal code|             Village|lat_center|lon_center|
+-----------+--------------------+----------+----------+
|       1000|             Brussel|  50.84275|   4.35155|
|       1000|           Bruxelles|  50.84275|   4.35155|
|       1005|Brusselse Hoofdst...| 50.844875| 4.3514333|
|       1005|Conseil Region Br...|  50.84786|  4.367408|
|       1008|Chambre des Repr�...| 50.846558|  4.364662|
|       1008|Kamer van Volksve...| 50.846558|  4.364662|
|       1009|   Senat de Belgique|  50.79834|   4.39565|
|       1010|Rijksadministrati...|  50.82433|  4.513954|
|       1012|Parlement de la C...| 50.846638| 4.3619924|
|       1020|             Brussel| 50.884216| 4.3580003|
|       1020|           Bruxelles| 50.884216| 4.3580003|
|       1020|              Laeken| 50.884216| 4.3580003|
|       1020|               Laken| 50.884216| 4.3580003|
|       1030|             Brussel|  50.86744|   4.37727|
|       1030|           Bruxell

## Stops

In [57]:
# Read in the file (as json) and convert to a single column
# The 'haltes' column gets renamed to 'stops'
stops = spark.read.json("data/stops.txt")
stops = stops.select((explode("haltes").alias("stops")))

# Map each entry in the dataframe to its own stop
stops = stops.select('stops').rdd.map(lambda x: x.stops).toDF()
# Drop unnecessary data
stops = stops.drop('links', 'gemeentenummer', 'entiteitnummer')

# Rename columns of dataframe to better name
stops = stops \
            .withColumnRenamed('haltenummer', 'stop_number') \
            .withColumnRenamed('omschrijving', 'desc') \
            .withColumnRenamed('geoCoordinaat', 'coord') \
            .withColumnRenamed('omschrijvingGemeente', 'vill') \

# Show the first 20 entries of the dataframe
stops.show()

+--------------------+-----------+--------------------+---------+
|               coord|stop_number|                desc|     vill|
+--------------------+-----------+--------------------+---------+
|[51.1638893702134...|     101000| A. Chantrainestraat|  Wilrijk|
|[51.2062496902375...|     101001|           Zurenborg|Antwerpen|
|[51.1660665941742...|     101002|Verenigde Natieslaan|  Hoboken|
|[51.1660216374063...|     101003|Verenigde Natieslaan|  Hoboken|
|[51.1740548394127...|     101004|     D. Baginierlaan|  Hoboken|
|[51.1630084393468...|     101005| A. Chantrainestraat|  Wilrijk|
|[51.1597748887066...|     101006|      Fotografielaan|  Wilrijk|
|[51.1599636330007...|     101007|      Fotografielaan|  Wilrijk|
|[51.1629556669243...|     101008|            Moerelei|  Wilrijk|
|[51.1634592883462...|     101009|            Moerelei|  Wilrijk|
|[51.1887431659368...|     101010|        J. De Voslei|Antwerpen|
|[51.1829725415369...|     101011|   Middelheim Vijver|Antwerpen|
|[51.16220

## Combination of the data

In [59]:
joined = stops.join(centers, stops.vill == centers.Village)
joined = joined.drop("vill")
joined.show()

+--------------------+-----------+--------------------+-----------+-------+----------+----------+
|               coord|stop_number|                desc|postal code|Village|lat_center|lon_center|
+--------------------+-----------+--------------------+-----------+-------+----------+----------+
|[50.7173104811086...|     208604|           Kamstraat|       1547|  Bever|  50.71827|  3.940144|
|[50.7148898550618...|     208606|              Burght|       1547|  Bever|  50.71827|  3.940144|
|[50.7122250352873...|     208607|            Kerkhove|       1547|  Bever|  50.71827|  3.940144|
|[50.7098088283453...|     208608|              Broeck|       1547|  Bever|  50.71827|  3.940144|
|[50.7034722690539...|     208609|             Commijn|       1547|  Bever|  50.71827|  3.940144|
|[50.6974143044988...|     208610|               Muydt|       1547|  Bever|  50.71827|  3.940144|
|[50.7283193498827...|     208613|            Akrenbos|       1547|  Bever|  50.71827|  3.940144|
|[50.7199986904738..

## Haversine function
A function to calculate the distance in between two geocoordinates. <br>
Found at https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

In [60]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## Calculate distance function

In [65]:
@udf(returnType=FloatType())
def calculate_distance(lat_center, lon_center, location):
    lat1 = lat_center
    lon1 = lon_center
    
    lat2 = location[0]
    lon2 = location[1]
    
    return haversine(lon1, lat1, lon2, lat2)

## Adding distances to dataframe

In [70]:
result = joined.withColumn('distance', 
                           calculate_distance('lat_center', 'lon_center', 'coord')).drop("coord", "lat_center", "lon_center")
result.show()

+-----------+--------------------+-----------+-------+----------+
|stop_number|                desc|postal code|Village|  distance|
+-----------+--------------------+-----------+-------+----------+
|     208604|           Kamstraat|       1547|  Bever|0.63004684|
|     208606|              Burght|       1547|  Bever|0.47692096|
|     208607|            Kerkhove|       1547|  Bever| 0.8597775|
|     208608|              Broeck|       1547|  Bever| 1.1527758|
|     208609|             Commijn|       1547|  Bever| 1.7655205|
|     208610|               Muydt|       1547|  Bever| 2.4481733|
|     208613|            Akrenbos|       1547|  Bever| 2.2808323|
|     208614|Akrenbos/Bloemendael|       1547|  Bever| 2.0864077|
|     208615|  Bloemendael/Burght|       1547|  Bever| 1.0272938|
|     209604|           Kamstraat|       1547|  Bever|0.64379346|
|     209606|              Burght|       1547|  Bever|0.50295454|
|     209607|            Kerkhove|       1547|  Bever|  0.891243|
|     2096